1. Przyjmij następujący zbiór danych wejściowych:
    1. bbb$
    2. aabbabd,
    3. ababcď
    4. abaababaabaabaabab$
    5. losowy tekst o długości 100 znaków,
    6. załączony plik.
2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.
3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.
4. Zaimplementuj algorytm konstruujący drzewo sufiksów.
5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem
szukiwania wzorców.
6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
    1. Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (lp.)/
    2. Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (lp.)
    3. Drzewo sufiksów - algorytm Ukkonena (3p).
7. Dla załączonego tekstu czas wariantów 1 i 2 może być nieakceptowalnie długi - w tej sytuacji pomiń wniki pomiarów dla tego tekstu
8. Oczekiwany wynik ćwiczenia to kod źródłowy oraz raport w formie PDF!


In [59]:
from random import randint
import time

1. Przyjmij następujący zbiór danych wejściowych:
    1. bbb$
    2. aabbabd,
    3. ababcď
    4. abaababaabaabaabab$
    5. losowy tekst o długości 100 znaków,
    6. załączony plik.
2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

In [53]:
s1 = "bbb$"
s2 = "aabbabd$"
s3 = "ababcd$"
s4 = "abaababaabaabaabab$"

s5 = "".join([chr(randint(97, 122)) for i in range(100)])
s5 += "$"

with open("1997_714_head.txt", "r") as f:
    s6 = f.read()
s6 += "$"

3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [98]:
class Node:
    def __init__(self, parent) -> None:
        self.parent = parent
        self.children = dict()

class Trie:
    def __init__(self, text) -> None:
        self.root = Node(None)
        for i in range(len(text)):
            self.add(text[i:], i)

    def add(self, text, index):
        node = self.root
        for i in range(len(text)):
            if text[i] not in node.children:
                node.children[text[i]] = Node(node)
            node = node.children[text[i]]
        node.children["$"] = index

    def find(self, pattern):
        node = self.root
        index = 0
        while index < len(pattern) and pattern[index] in node.children:
            node = node.children[pattern[index]]
            index += 1
        return node, index
    
    def pattern_search(self, pattern):
        if len(pattern) == 0:
            return True
        node, index = self.find(pattern)
        return node.children != {} and index == len(pattern)

class Trie2:
    def __init__(self, text) -> None:
        self.root = Node(None)
        for i in range(len(text)):
            self.add(text[i:])
        
    def add(self, text):
        node = self.root
        for i in range(len(text)):
            if text[i] not in node.children:
                node.children[text[i]] = Node(node)
            node = node.children[text[i]]
        node.children["$"] = True
    

    def find(self, pattern):
        node = self.root
        index = 0
        while index < len(pattern) and pattern[index] in node.children:
            node = node.children[pattern[index]]
            index += 1
        return node, index
    
    def pattern_search(self, pattern):
        if len(pattern) == 0:
            return True
        node, index = self.find(pattern)
        return node.children != {} and index == len(pattern)

4. Zaimplementuj algorytm konstruujący drzewo sufiksów.

In [55]:
class SuffixNode:
    def __init__(self, start, end) -> None:
        self.start = start
        self.end = end
        self.children = dict()

class SuffixTree:
    def __init__(self, text) -> None:
        self.root = SuffixNode(0, len(text) - 1)
        self.text = text
        for i in range(len(text) - 1):
            head, depth = self.find(text[i:])
            self.add(head, depth, i)

    def add(self, node, depth, index):
        new_node = SuffixNode(index + depth, len(self.text) - 1)
        node.children[self.text[new_node.start]] = new_node


    def find(self, text, depth=0, node=None):
        if node is None:
            node = self.root
        next_node = node.children.get(text[0])
        if next_node is None:
            return node, depth

        next_node_text_len = next_node.end - next_node.start + 1
        for i in range(1, next_node_text_len):
            if self.text[next_node.start + i] != text[i]:
                stop_node = SuffixNode(next_node.start, next_node.start + i - 1)
                next_node.start += i
                node.children[self.text[stop_node.start]] = stop_node
                stop_node.children[self.text[next_node.start]] = next_node
                return stop_node, depth + i
        return self.find(text[next_node_text_len:], next_node_text_len + depth, next_node)
    
    def pattern_search(self, pattern, node=None):
        if len(pattern) == 0:
            return True
        if node is None:
            node = self.root
        next_node = node.children.get(pattern[0])
        if next_node is None:
            return False

        next_node_text_len = next_node.end - next_node.start + 1
        for i in range(1, next_node_text_len):
            if i == len(pattern):
                return True
            elif self.text[next_node.start + i] != pattern[i]:
                return False
        return self.pattern_search(pattern[next_node_text_len:], next_node)

5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem
szukiwania wzorców.

In [56]:
texts = [s1, s2, s3, s4, s5, s6]

test_patterns = ["pietrusza", "maslo", "qwerty", "jjjj", "halapenia", "marchefka"]

errors = 0
print("============= TRIE =============")
for i, text in enumerate(texts):
    trie = Trie(text)
    t_er = 0
    for pattern in test_patterns:
        if trie.pattern_search(pattern):
            print("Error:", text, pattern)
            errors += 1
            t_er += 1
    if t_er == 0:
        print("text ", i+1, ": OK", sep="")
print("Found", errors, "errors\n")

errors = 0
print("========= SUFFIX TREE ==========")
for i, text in enumerate(texts):
    suffix_tree = SuffixTree(text)
    t_er = 0
    for pattern in test_patterns:
        if suffix_tree.pattern_search(pattern):
            print("Error:", text, pattern)
            errors += 1
            t_er += 1
    if t_er == 0:
        print("text ", i+1, ": OK", sep="")
print("Found", errors, "errors")

============= TRIE =============
text 1: OK
text 2: OK
text 3: OK
text 4: OK
text 5: OK
text 6: OK
Found 0 errors

========= SUFFIX TREE ==========
text 1: OK
text 2: OK
text 3: OK
text 4: OK
text 5: OK
text 6: OK
Found 0 errors


In [57]:
print("============= TRIE =============")
print("text 1, pattern 'bb':            ", Trie(s1).pattern_search("bb"))
print("text 2, pattern 'aab':           ", Trie(s2).pattern_search("aab"))
print("text 3, pattern 'abc':           ", Trie(s3).pattern_search("abc"))
print("text 4, pattern 'aabaabaabab':   ", Trie(s4).pattern_search("aabaabaabab"))
print("text 6, pattern 'Dz.U. z':       ", Trie(s6).pattern_search("Dz.U. z"))

print("\n========= SUFFIX TREE ==========")
print("text 1, pattern 'bb':            ", SuffixTree(s1).pattern_search("bb"))
print("text 2, pattern 'aab':           ", SuffixTree(s2).pattern_search("aab"))
print("text 3, pattern 'abc':           ", SuffixTree(s3).pattern_search("abc"))
print("text 4, pattern 'aabaabaabab':   ", SuffixTree(s4).pattern_search("aabaabaabab"))
print("text 6, pattern 'Dz.U. z':       ", SuffixTree(s6).pattern_search("Dz.U. z"))

============= TRIE =============
text 1, pattern 'bb':             True
text 2, pattern 'aab':            True
text 3, pattern 'abc':            True
text 4, pattern 'aabaabaabab':    True
text 6, pattern 'Dz.U. z':        True

========= SUFFIX TREE ==========
text 1, pattern 'bb':             True
text 2, pattern 'aab':            True
text 3, pattern 'abc':            True
text 4, pattern 'aabaabaabab':    True
text 6, pattern 'Dz.U. z':        True


6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
    1. Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (lp.)/
    2. Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (lp.)
    3. Drzewo sufiksów - algorytm Ukkonena (3p).

In [104]:
def time_test(method, text, text_num, pattern):
    start = time.perf_counter()
    method(text).pattern_search(pattern)
    end = time.perf_counter()
    print("text", text_num, "pattern", pattern, "time:", end - start)

In [105]:
print("\n============= TRIE =============")

time_test(Trie, s1, 1, "bb")
time_test(Trie, s2, 2, "aab")
time_test(Trie, s3, 3, "abc")
time_test(Trie, s4, 4, "aabaabaabab")
time_test(Trie, s6, 6, "Dz.U. z")

print("\n============ TRIE2 =============")

time_test(Trie2, s1, 1, "bb")
time_test(Trie2, s2, 2, "aab")
time_test(Trie2, s3, 3, "abc")
time_test(Trie2, s4, 4, "aabaabaabab")
time_test(Trie2, s6, 6, "Dz.U. z")

print("\n========= SUFFIX TREE ==========")

time_test(SuffixTree, s1, 1, "bb")
time_test(SuffixTree, s2, 2, "aab")
time_test(SuffixTree, s3, 3, "abc")
time_test(SuffixTree, s4, 4, "aabaabaabab")
time_test(SuffixTree, s6, 6, "Dz.U. z")


============= TRIE =============
text 1 pattern bb time: 3.619998460635543e-05
text 2 pattern aab time: 4.229997284710407e-05
text 3 pattern abc time: 3.060000017285347e-05
text 4 pattern aabaabaabab time: 0.0002932000206783414
text 6 pattern Dz.U. z time: 9.259265299944673

============ TRIE2 =============
text 1 pattern bb time: 1.4399993233382702e-05
text 2 pattern aab time: 1.92999723367393e-05
text 3 pattern abc time: 1.5900004655122757e-05
text 4 pattern aabaabaabab time: 9.180000051856041e-05
text 6 pattern Dz.U. z time: 10.634293699986301

========= SUFFIX TREE ==========
text 1 pattern bb time: 2.300000051036477e-05
text 2 pattern aab time: 1.5700003132224083e-05
text 3 pattern abc time: 1.1200027074664831e-05
text 4 pattern aabaabaabab time: 4.829996032640338e-05
text 6 pattern Dz.U. z time: 0.020694000006187707
